In [ ]:
import glob
import pandas as pd

relacion = []

In [ ]:
a7 = pd.read_csv('AVANCE_7_.csv', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})

a7.head()

In [ ]:
pd.crosstab(a7['FUNNEL_4'], a7['PERIODO'])

In [ ]:
print(a7.shape)
a7 = a7[a7['FUNNEL_4'] == 1]
print(a7.shape)

In [ ]:
cols_up = ['COD_UNICO', 'PERIODO', 'MONTOUTILIZADO', 'NOMBRE_COMPLETO', 'FEVE', 'MAX_DIAS_ATRASO_SF', 'MAX_DIAS_ATRASO', 'DES_GRUPO_ECONOMICO',
           'SUBSECTOR', 'SEI_MCDO', 'GRUPO_ECON_FEVE_SEGUIR', 'MONTO_PROTESTO_U1M','MONTO_PROTESTO_U6M' , 'MONTO_PROTESTO_U12M',
           'GRUPO_ECON_FEVE_DURO',  'GRUPO_ECON_CLASIF_NO_NORMAL', 'MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF',
           'MTO_DEUDA_VENCIDO_COLDIRECT_SIN_REATCIVA_SF', 'MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF',
           'RIESGO', 'BUCKET', 'COL_DIRECTA_MES_ACTUAL', 'RIESGO_DIRECTO_CIERRE_MES', 'RIESGO_INDIRECTO_CIERRE_MES',
           'TIENE_DEUDA_REFINANCIADO_RCC', 'TIENE_DEUDA_REFINANCIADO_IBK', 'ZONAL', 'SEGMENTO', 'CLASIF_SBS',
           'TIENE_DEUDA_JUDICIAL_RCC', 'DEBT_COMEX_VIGENTE_PERCENT_ULT_RCC', 'DEBT_COMEX_VENCIDO_PERCENT_ULT_RCC',
           'PERIODO_CEF_ANTERIOR','PERIODO_ULTIMO_CEF', 'ALERTAS_ACTIVAS', 'ALERTAS_MES_ANTERIOR', 'MISMAS_ALERTAS',
           'MTO_ATRASO_NOLETRAS', 'DEUDA_TOTAL_RCC']

len(cols_up)

In [ ]:
#target_FINAL_fusion_atraso_feve_2Mbueno

#target_seguircovid_reducir_salir_2Mbueno  = 0
#target_rcc_8_dias_atraso_2Mbueno = 1
#riesgo > 150 000

In [ ]:
cols_target = [col for col in a7.columns if 'target' in col.lower() and ('2m' in col.lower() or '30' in col.lower() or 'final' in col.lower() )]
print(cols_target)

In [ ]:
for col in cols_target:
    display(
        pd.crosstab(
            a7[(a7['PERIODO'] == '202101') & (a7['target_FINAL_fusion_atraso_feve_2Mbueno'].isnull())]['ZONAL'].fillna(-1), 
            a7[(a7['PERIODO'] == '202101') & (a7['target_FINAL_fusion_atraso_feve_2Mbueno'].isnull())][col].fillna(-1))
    )

In [ ]:
cols_alertas_63 = [_ for _ in a7.columns if 'NEW' in _.upper()]
print(cols_alertas_63, len(cols_alertas_63))


cols_alertas_63_new = ['A{}_NEW'.format(_ + 1) for _ in range(66)] + ['A_ME_NEW', 'A_GE_NEW', 'A_P1_NEW', 'A_P2_NEW'] ## 21 ALERTAS GE
print(cols_alertas_63_new)  ## 66 ALERTAS

In [ ]:
cols_alertas_63 = cols_alertas_63 + cols_alertas_63_new.copy() # cols_alertas_63 + cols_alertas_63_new
len(cols_alertas_63)

In [ ]:
lambda_alertado = lambda _: 'alertado' if _ > 0 else 'no alertado'

a7['ALERTADO_ME_NEW'] = a7['A_ME_NEW'].apply(lambda_alertado)
a7['ALERTADO_GE_NEW'] = a7['A_GE_NEW'].apply(lambda_alertado)
a7['ALERTADO_P1_NEW'] = a7['A_P1_NEW'].apply(lambda_alertado)
a7['ALERTADO_P2_NEW'] = a7['A_P2_NEW'].apply(lambda_alertado)

cols_alertas_63 = cols_alertas_63 + ['ALERTADO_ME_NEW', 'ALERTADO_GE_NEW', 'ALERTADO_P1_NEW', 'ALERTADO_P2_NEW']
cols_alertas_63 = list(set(cols_alertas_63))

len(cols_alertas_63)

In [ ]:
cols_sql = list(set(cols_up + cols_target + cols_alertas_63 ))
len(cols_sql)

In [ ]:
for col in ['GRUPO_ECON_FEVE_SEGUIR', 'GRUPO_ECON_FEVE_DURO',  'GRUPO_ECON_CLASIF_NO_NORMAL']:
    a7[col] = a7[col].fillna("-")
    
a7['MONTO_PROTESTO_U1M'] = a7['PROTESTOS_U1M'].copy()
a7['MONTO_PROTESTO_U6M'] = a7['PROTESTOS_U6M'].copy()
a7['MONTO_PROTESTO_U12M'] = a7['PROTESTOS_U12M'].copy()

for col in ['MONTO_PROTESTO_U1M','MONTO_PROTESTO_U6M' , 'MONTO_PROTESTO_U12M', 
            'MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF', 'MTO_DEUDA_VENCIDO_COLDIRECT_SIN_REATCIVA_SF', 
           'MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF']:

        a7[col] = a7[col].fillna(0)

In [ ]:
a7[cols_sql].shape

In [ ]:
print(cols_sql)

In [ ]:
a7[cols_sql].to_csv('AVANCE_8_.csv', sep='|', index=False)

In [ ]:
a7[a7['MISMAS_ALERTAS'] == 1][
    ['ALERTAS_ACTIVAS', 'ALERTAS_MES_ANTERIOR', 'MISMAS_ALERTAS', 'GRUPO_ECON_FEVE_SEGUIR', 'GRUPO_ECON_FEVE_DURO',  'GRUPO_ECON_CLASIF_NO_NORMAL', 'DES_GRUPO_ECONOMICO']
]

In [ ]:
a7[cols_sql]['PERIODO'].value_counts(dropna=False)

In [ ]:
%%time
# mssql+pymssql

from sqlalchemy import create_engine
engine = create_engine(r"mssql+pymssql://ibetlmarket:m@rk3t2o15@10.11.12.90\BDT/MARKET")
connection_x = engine.connect()

try:
    a7[cols_sql].to_sql(
        name='FEVE_AJUSTADO_ALERTAS_TARGETS',
        schema='ODS',
        con=connection_x, 
        index=False, 
        if_exists='replace'
    )
    connection_x.close()
    correct = 1

except Exception as e:
    print("ERROR: ", str(e))
    correct = 0

print(correct)

In [ ]:
connection_x.close()
a7[cols_sql]['PERIODO'].value_counts()

In [ ]:
a7.shape

In [ ]:
#import pandas as pd
#data = pd.read_csv('https://cloud.minsa.gob.pe/s/ZgXoXqK2KLjRLxD/download')
#data.shape

In [ ]:
#data.shape

<iframe width="600" height="373.5" src="https://app.powerbi.com/view?r=eyJrIjoiZDNmOWJiMWMtYTRlYS00ZGE3LWJjZTEtMGFlODRlOWMxYzc0IiwidCI6ImM0YTY2YzM0LTJiYjctNDUxZi04YmUxLWIyYzI2YTQzMDE1OCIsImMiOjR9" frameborder="0" allowFullScreen="true"></iframe>